In [66]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Read the CSV file into a DataFrame
titanic_df = pd.read_csv('./Resources/Titanic CSV.csv')

# Display the first 5 rows
titanic_df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [55]:
# Drop rows with missing values and reset the index
titanic_df = titanic_df.dropna().reset_index(drop=True)
titanic_df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
2,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
3,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
4,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S


In [56]:
# Count the number of rows in the DataFrame
len(titanic_df) 

183

In [57]:
# Create the passengers, tickets, and cabins dataframes and remove index
passengers_df = titanic_df[['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked']]
tickets_df = titanic_df[['PassengerId', 'Ticket', 'Fare']]
cabins_df = titanic_df[['PassengerId', 'Cabin']]


In [44]:
# save the dataframes to CSV files
passengers_df.to_csv('passengers.csv', index=False)
tickets_df.to_csv('tickets.csv', index=False)
cabins_df.to_csv('cabins.csv', index=False)


In [69]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect("titanic_db.sqlite")
cursor = conn.cursor()

In [70]:
# Create passengers table
cursor.execute('''
CREATE TABLE passengers (
    PassengerId INTEGER PRIMARY KEY,
    Survived INTEGER,
    Pclass INTEGER,
    Name TEXT,
    Sex TEXT,
    Age REAL,
    SibSp INTEGER,
    Parch INTEGER,
    Embarked TEXT
);
''')

# Create tickets table
cursor.execute('''
CREATE TABLE tickets (
    PassengerId INTEGER PRIMARY KEY,
    Ticket TEXT,
    Fare REAL,
    FOREIGN KEY (PassengerId) REFERENCES passengers(PassengerId)
);
''')

# Create cabins table
cursor.execute('''
CREATE TABLE cabins (
    PassengerId INTEGER PRIMARY KEY,
    Cabin TEXT,
    FOREIGN KEY (PassengerId) REFERENCES passengers(PassengerId)
);
''')

In [71]:
# Load data from CSV files into tables
def load_csv_to_db(csv_path, table_name):
    df = pd.read_csv(csv_path)
    df.to_sql(table_name, conn, if_exists='append', index=False)

load_csv_to_db('C:/Users/joe/Project3_Practice/passengers.csv', 'passengers')
load_csv_to_db('C:/Users/joe/Project3_Practice/tickets.csv', 'tickets')
load_csv_to_db('C:/Users/joe/Project3_Practice/cabins.csv', 'cabins')

# Verify data loading
for table in ['passengers', 'tickets', 'cabins']:
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    print(f"Total records in {table}: ", cursor.fetchone()[0])

Total records in passengers:  183
Total records in tickets:  183
Total records in cabins:  183


In [62]:
# Display first 5 rows from passengers table in a pandas DataFrame
passengers_df = pd.read_sql_query("SELECT * FROM passengers LIMIT 5;", conn)
passengers_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Embarked
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,C
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,S
2,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,S
3,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,S
4,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,S


In [67]:
# Delete the tables
cursor.execute("DROP TABLE IF EXISTS passengers;")
cursor.execute("DROP TABLE IF EXISTS tickets;")
cursor.execute("DROP TABLE IF EXISTS cabins;")
conn.commit()   

In [68]:
# Commit and close connection
conn.commit()
conn.close()


Additional Library Not Learned in Class

In [45]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

# 1. Load the Data
df = pd.read_csv("Titanic CSV.csv")

# 2. Prepare the Data
# Handle missing values
df['Age'] = df['Age'].fillna(df['Age'].median())
df.dropna(subset=['Embarked'], inplace=True)  # Remove rows with missing Embarked

# Select features
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
target = 'Survived'

# Encode categorical variables
transformer = make_column_transformer(
    (OneHotEncoder(), ['Sex', 'Embarked']),
    remainder='passthrough')

transformed_features = transformer.fit_transform(df[features])
transformed_feature_names = transformer.get_feature_names_out(input_features=features)

X = pd.DataFrame(transformed_features, columns=transformed_feature_names)
y = df[target]

# 3. Split the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Create a Model
model = LogisticRegression(solver='liblinear', random_state=42)

# 5. Train the Model
model.fit(X_train, y_train)

# 6. Make Predictions
y_pred = model.predict(X_test)

# 7. Evaluate the Model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7808988764044944


In [46]:
# print the model coefficients
coefficients = model.coef_[0]
print("Model Coefficients:")
for feature, coef in zip(X.columns, coefficients):
    print(f"{feature}: {coef:.4f}")

Model Coefficients:
onehotencoder__Sex_female: 2.2467
onehotencoder__Sex_male: -0.3974
onehotencoder__Embarked_C: 0.8462
onehotencoder__Embarked_Q: 0.6416
onehotencoder__Embarked_S: 0.3614
remainder__Pclass: -0.9758
remainder__Age: -0.0368
remainder__SibSp: -0.3628
remainder__Parch: -0.0706
remainder__Fare: 0.0032


In [ ]:
# Display first 5 rows from passengers table in a pandas DataFrame
passengers_df = pd.read_sql_query("SELECT * FROM passengers LIMIT 5;", conn)
passengers_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Embarked
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,C
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,S
2,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,S
3,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,S
4,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,S
